In [ ]:
from PIL import Image
from datetime import datetime
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import BatchNormalization, Flatten, Activation, GlobalAveragePooling2D, Lambda
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from sklearn.metrics import classification_report, confusion_matrix
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import sklearn
import tensorflow as tf
import warnings

In [ ]:
BATCH_SIZE = 64
NUM_CLASS = 200
train_dir = "/kaggle/input/ucsd-200/Train/Train"
test_dir = "/kaggle/input/ucsd-200/Test/Test"
train_txt_path = '/kaggle/input/ucsd-200/train.txt'
test_txt_path = '/kaggle/input/ucsd-200/test.txt'

#Reading the txt file
train_data, test_data = [], []
with open(train_txt_path, 'r') as train_file:
  train_data = train_file.read().splitlines()
with open(test_txt_path, 'r') as test_file:
  test_data = test_file.read().splitlines()

train_df = pd.DataFrame([x.split() for x in train_data], columns = ['filename', 'class'])
test_df = pd.DataFrame([x.split() for x in test_data], columns = ['filename', 'class'])

train_df = train_df.sort_values('filename').reset_index(drop=True)
test_df = test_df.sort_values('filename').reset_index(drop=True)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,  
    zoom_range=0.2,  
    horizontal_flip=True,  
    rotation_range=20,  
    width_shift_range=0.2,
    height_shift_range=0.2,  
)


test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_augmented = train_datagen.flow_from_dataframe(
    train_df.sort_values('filename').reset_index(drop=True), 
    directory=train_dir,
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True  
)


test_augmented = test_datagen.flow_from_dataframe(
    test_df.sort_values('filename').reset_index(drop=True),  
    directory=test_dir,
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False  
)

In [ ]:
species_names = {v: k for k, v in train_augmented.class_indices.items()}

images, labels = next(train_augmented)

plt.figure(figsize=(20, 20))

for i in range(25): 
    ax = plt.subplot(5, 5, i + 1)
    
    img = images[i] * 255.0
    plt.imshow(img.astype(np.uint8))  

    label_index = np.argmax(labels[i])
    
    plt.title(species_names[label_index])  
    plt.axis('off')  

plt.show()

In [ ]:
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, Concatenate
from tensorflow.keras.applications import ResNet50, MobileNetV2, EfficientNetB0
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.callbacks import ReduceLROnPlateau

IMG_SHAPE = (224, 224, 3)

resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)
mobilenet_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)
efficientnet_base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)

for layer in resnet_base.layers[:-3]:
    layer.trainable = True
for layer in mobilenet_base.layers[:-3]:
    layer.trainable = True
for layer in efficientnet_base.layers[:-3]:
    layer.trainable = True

input_layer = Input(shape=IMG_SHAPE)

resnet_output = resnet_base(input_layer)
mobilenet_output = mobilenet_base(input_layer)
efficientnet_output = efficientnet_base(input_layer)

resnet_output = GlobalAveragePooling2D()(resnet_output)
mobilenet_output = GlobalAveragePooling2D()(mobilenet_output)
efficientnet_output = GlobalAveragePooling2D()(efficientnet_output)

combined_output = Concatenate()([resnet_output, mobilenet_output, efficientnet_output])

x = Dropout(0.45)(combined_output)

output_layer = Dense(200, activation='softmax', kernel_regularizer=l2(0.02), kernel_constraint=max_norm(3))(x)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy',nmetrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5,  restore_best_weights=True,  verbose=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=1e-8, verbose=1)

model.summary()

train_model = model.fit(
    train_augmented,
    steps_per_epoch=train_augmented.samples, 
    validation_data=test_augmented,
    validation_steps=test_augmented.samples,
    callbacks = [reduce_lr,early_stopping],
    epochs = 200
)

In [ ]:
# Plotting training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(train_model.history['accuracy'], label='Training Accuracy')
plt.plot(train_model.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1]) 
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
train_model = model.fit(
    train_augmented,
    steps_per_epoch=train_augmented.samples,  
    validation_data=test_augmented,
    validation_steps=test_augmented.samples,
    callbacks = [reduce_lr,early_stopping],
    epochs = 200
)

In [ ]:
# Plotting training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(train_model.history['accuracy'], label='Training Accuracy')
plt.plot(train_model.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1]) 
plt.legend(loc='lower right')
plt.grid(True)
plt.show()